In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
np.random.seed(30300101)
nLives = 100

In [3]:
#Storage of variables used to define the dataset
#Start with policy variables

#4 products, to be defined later
prods = range(4)
prods = [x + 1 for x in prods]
prodsWt = [0.5,0.25,0.2,0.05]

#Evenly distribute business within issue year confines
issYrsStart = 2008
issYrs = range(10)
issYrs = [x + issYrsStart for x in issYrs]
issYrsWt = [1/len(issYrs)] * len(issYrs)

#Randomly distribute business amongst n states
#Given assignment methodology, possible for multiple states to be picked during the same run
nIssSt = 10
issSt = range(50)
issSt = [x + 1 for x in issSt]
issStWt = [0] * len(issSt)
st = np.random.randint(low=0,high=len(issSt)-1,size=nIssSt).tolist()
rd = np.random.uniform(0,1,nIssSt).tolist()
rd = [x/sum(rd) for x in rd]
for i in range(len(st)):
    issStWt[st[i]] = issStWt[st[i]] + rd[i]

In [4]:
#Next to coverage variables

#Future state utilize product mapping table to enable/disable coverage riders at a product level
#Start with random probability for now and a simple assignment of base/rider types
covCnt = [0,1]
covCntWt = [0.9,0.1]

#Presume all riders are optioned at issue

In [5]:
#Next to person variables

#Randomly distribute ages within an issue age band range
#Will be used to calculate dates of birth
#Currently presume an ALB issue age calculation
age = [0,25,35,45,55,65,75,999]
ageWt = [0,0.05,0.15,0.25,0.3,0.15,0.1,0]

#Two genders within underwriting criteria
sex = range(2) 
sexWt = [0.5,0.5]

#Spousal riders
#Current code presumes no joint life policies, and that lives on coverages match back to base
#Future state may contemplate assigning a different insured on rider records, or joint life policies
lives = [0,1,2]
livesWt = [0,1,0]

#Multiple issues
multiplePol = [0,1,2,3,4,5]
multiplePolWt = [0,0.8,0.1,0.05,0.025,0.025]

In [6]:
#Finally, temporal statistics

#Probability of lapse
lapseDur = [1,2,3,99]
lapseWt = [0.2,0.1,0.05,0.65]
ultLapse = 0.05

#Probability of death
#Modeled as an exponential
#Import SOA mortality table and lookup?

In [7]:
#UDFs
def assignLabel(rnd, labelArray,weightArray):
    c = 0
    for i in range(len(weightArray)):
        c = c + weightArray[i]
        if c > rnd:
            x = labelArray[i]
            break
    return x

def rndDate(rnd,start,end):
    fmt = '%Y-%m-%d'
    #st = datetime.strptime(start,fmt).timestamp()
    #ed = datetime.strptime(end,fmt).timestamp()
    st = datetime.datetime.strptime(start,fmt)
    ed = datetime.datetime.strptime(end,fmt)
    dt = st + rnd * (ed - st)
    #dt = datetime.utcfromtimestamp(dt).strftime(fmt)
    dt = datetime.datetime.strftime(dt,fmt)
    return dt

def rndGammaDate(start,shape,scale):
    fmt = '%Y-%m-%d'
    st = datetime.datetime.strptime(start,fmt)
    dt = st + datetime.timedelta(days = np.random.gamma(shape,scale) * 365)
    dt = datetime.datetime.strftime(dt,fmt)
    return dt

In [ ]:
#Person creation
lifedf = pd.DataFrame(columns=['lifeId','name','productId','issueDate','issueState'])
for i in range(nPols):
    polId = i + 1
    polNo = hex(hash(np.random.uniform(0,1)))[0:10]
    prodId = assignLabel(np.random.uniform(0,1),prods,prodsWt)
    issueYr = assignLabel(np.random.uniform(0,1),issYrs,issYrsWt)
    issueDt = rndDate(np.random.uniform(0,1),str(issueYr)+'-01-01',str(issueYr)+'-12-31')
    issStId = assignLabel(np.random.uniform(0,1),issSt,issStWt)
    poldf.loc[i] = [polId,polNo,prodId,issueDt,issStId]